In [1]:
import pandas as pd
import numpy as np
import time

import re

from bs4 import BeautifulSoup

In [2]:
# 셀레니움 import
from selenium import webdriver # 크롬을 자동화 시켜서 제어할 수 있는 프로그램
                               # 그걸 셀레니움으로 컨트롤
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # 자동설정에 필요한 함수들
from selenium.webdriver.common.keys import Keys # 입력키

In [3]:
# 명시적 대기(explicit wait)
from selenium.webdriver.common.by import By # 셀렉터 기준 (어떤 기준으로 찾을지 명시)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# warning ignore
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
# webdriver 자동설정
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [6]:
# 다운받은 웹 드라이버 경로를 문자열로 크롬함수에 전달
driver = webdriver.Chrome('./chromedriver.exe')

In [7]:
# url 명시
url = 'https://www.ratebeer.com/search?tab=beer'

In [8]:
#######리뷰 따로 크롤링######
### 추가 맥주 리스트 포함 최종 크롤링 (제발...) ###


beer_list = ['kloud', 'fitz super clear', 'Asahi super dry', 'Tsingtao', 'Heineken',
 'Kirin ichiban', 'Sapporo Premium Beer / Draft Beer', 'stella artois', 'guinness braught',
 '1664 Blanc', 'pilsner urquell', 'San Miguel','OB premier pilsner', 'cass fresh',
 'Hite Black Beer Stout', 'dry finish', 'max hite', 'hite extra cold', 'victoria bitter', 'BINTANG pilsner',
 'krombacher weizen', 'Miller Genuine Draft', 'Hoegaarden Cherry', 'TIGER REDLER',
 "Suntory The Premium Malt's", 'REEPER B Weissbier', 'PEEPER B IPA', 'TIGER BEER',
 'TSINGTAO WHEAT BEER', 'Erdinger Weissbier', 'Carlsberg', 'Budweiser ', 'Hoegaarden',
 'YEBISU', 'Paulaner Hefe-weissbier', 'Desperados', 'Peroni Nastro Azzurro',
 'Edelweiss Snowfresh', 'Heineken Dark Lager', 'Kozel Dark 10', 'Guinness original',
 'Filite', 'SEOULITE ALE', 'JEJU WIT ALE', 'Stephans Brau Philsner', 'Stephans Brau Larger',
 'Stephans Bräu Hefe-Weizen Naturtrüb', 'Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type", 'Terra', 
 'Hite Zero', 'HiteJinro Filite Fresh Cool & Clear', 'Cass Light', 'Happoshu Filgood', 
 'Kloud Draft Original Gravity', 'Hite Beer', '7Bräu Gompyo Wheatbeer', 
 'Moonbear Geumgangsan Golden Ale','Blue Moon Belgian White', 'Heineken Silver', 'Cafri']

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #     options = webdriver.ChromeOptions()
    #     options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    # 명시적 대기 - XPATH를 이용한 경우
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')))
    #time.sleep(4)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input').send_keys(beer)
    #time.sleep(4)

    # 2번 사진에 해당 : 상품 선택
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]')))
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4')))
    beer_name = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    error_cnt = 0
    print(beer_name)
    while 1:
        try :
            # 4번 사진에 해당 : 전체 리뷰 개수 수집
            time.sleep(4)
            # elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6')))
            string = driver.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text
            print(string)

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')))
    # element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    #time.sleep(4)

    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    driver.execute_script("arguments[0].click();", elem)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')
        # Show more
        #driver.execute_script("arguments[0].click();", element)
        show_more = driver.find_elements(By.CSS_SELECTOR,'section.BeerReviewListItem___StyledSection2-gzaqoT.JrTsK > div.fj-s.f-wrap > div > button > span.MuiButton-label')
        for i in range(len(show_more)):
            driver.execute_script("arguments[0].click();", show_more[i])

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        # 맥주 정보
        beer_info = driver.find_elements(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap')
        beer_review = driver.find_elements(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.colorized__WrappedComponent-hrwcZr.hwjOn.BeerReviewListItem___StyledText-kMbsdb.gCtEHi.pre-wrap.MuiTypography-body1')
        tmp = []
        tmp_review = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        for i in range(len(beer_review)):
            tmp_review.append(beer_review[i].text)
            # print(tmp_review.append(beer_review[i].text))
            
        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['검색이름'] = beer
        tmp['beer_review'] = tmp_review

        data = pd.concat([data, tmp])
        # data = pd.concat([data, tmp_review], axis = 1).reset_index(inplace=True, drop=True)

        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(2)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result

In [11]:
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

url_open... kloud 맥주 데이터를 수집합니다..
Kloud Original Gravity
Reviews (108)
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.


In [114]:
#######리뷰 따로 크롤링######
### 추가 맥주 리스트 포함 최종 크롤링 (제발...) ###


beer_list = ['hite zero']

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #     options = webdriver.ChromeOptions()
    #     options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    # 명시적 대기 - XPATH를 이용한 경우
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')))
    #time.sleep(4)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input').send_keys(beer)
    #time.sleep(4)

    # 2번 사진에 해당 : 상품 선택
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]')))
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4')))
    beer_name = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    error_cnt = 0
    print(beer_name)
    while 1:
        try :
            # 4번 사진에 해당 : 전체 리뷰 개수 수집
            time.sleep(4)
            # elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6')))
            string = driver.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text
            print(string)

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')))
    # element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    #time.sleep(4)

    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    driver.execute_script("arguments[0].click();", elem)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')
        # Show more
        #driver.execute_script("arguments[0].click();", element)
        show_more = driver.find_elements(By.CSS_SELECTOR,'section.BeerReviewListItem___StyledSection2-gzaqoT.JrTsK > div.fj-s.f-wrap > div > button > span.MuiButton-label')
        for i in range(len(show_more)):
            driver.execute_script("arguments[0].click();", show_more[i])

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        # 맥주 정보
        beer_info = driver.find_elements(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap')
        beer_review = driver.find_elements(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.colorized__WrappedComponent-hrwcZr.hwjOn.BeerReviewListItem___StyledText-kMbsdb.gCtEHi.pre-wrap.MuiTypography-body1')
        tmp = []
        tmp_review = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        for i in range(len(beer_review)):
            tmp_review.append(beer_review[i].text)
            # print(tmp_review.append(beer_review[i].text))
            
        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['검색이름'] = beer
        tmp['beer_review'] = tmp_review

        data = pd.concat([data, tmp])
        # data = pd.concat([data, tmp_review], axis = 1).reset_index(inplace=True, drop=True)
        print(data)
        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(2)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result

In [112]:
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

url_open... hite zero 맥주 데이터를 수집합니다..
Hite Zero
Reviews (12)
성공... while문을 탈출합니다.
1 번째 페이지입니다.
                                            beer_info  beer_name       검색이름  \
0   Stew41(2,523)\n🇦🇺Melbourne, Australia\n2.6Augu...  Hite Zero  hite zero   
1   alex_leit(14,300)\n🇧🇾Minsk, Belarus\n0.9August...  Hite Zero  hite zero   
2   BarrelTapper31(190)\n🇷🇺Russia\n2.0February 7, ...  Hite Zero  hite zero   
3   kdd230(107)\n🇬🇧Carshalton, England\n1.9May 11,...  Hite Zero  hite zero   
4   Rubin77(7,853)\n🇧🇪Brussels, Belgium\n2.1Februa...  Hite Zero  hite zero   
5   YantarCoast(4,011)\n🇷🇺Moscow, Russia\n1.4Janua...  Hite Zero  hite zero   
6   nerdboy19(171)\n🇰🇷Seoul, South Korea\n2.4Janua...  Hite Zero  hite zero   
7   ycsabu(5)\n🇰🇷South Korea\n0.9January 4, 2016\n...  Hite Zero  hite zero   
8   fredthecat(548)\n🇨🇦Canada\n0.8April 26, 2015\n...  Hite Zero  hite zero   
9   FatPhil(23,306)\n🇪🇪Tallinn, Estonia\n1.4April ...  Hite Zero  hite zero   
10  calix(1)\n5.0January 31, 2014\nO

In [11]:
#######리뷰 따로 크롤링######
### 추가 맥주 리스트 포함 최종 크롤링 (제발...) ###


beer_list = ['Berliner Kindl Pilsener']

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['beer_info', '검색이름', 'beer_name'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #     options = webdriver.ChromeOptions()
    #     options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    # 명시적 대기 - XPATH를 이용한 경우
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')))
    #time.sleep(4)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input').send_keys(beer)
    #time.sleep(4)

    # 2번 사진에 해당 : 상품 선택
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]')))
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4')))
    beer_name = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    error_cnt = 0
    print(beer_name)
    while 1:
        try :
            # 4번 사진에 해당 : 전체 리뷰 개수 수집
            time.sleep(4)
            # elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6')))
            string = driver.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text
            print(string)

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')))
    # element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    #time.sleep(4)

    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    driver.execute_script("arguments[0].click();", elem)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')
        # Show more
        #driver.execute_script("arguments[0].click();", element)
        show_more = driver.find_elements(By.CSS_SELECTOR,'section.BeerReviewListItem___StyledSection2-gzaqoT.JrTsK > div.fj-s.f-wrap > div > button > span.MuiButton-label')
        for i in range(len(show_more)):
            driver.execute_script("arguments[0].click();", show_more[i])

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        # 맥주 정보
        beer_info = driver.find_elements(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap')
        beer_review = driver.find_elements(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.colorized__WrappedComponent-hrwcZr.hwjOn.BeerReviewListItem___StyledText-kMbsdb.gCtEHi.pre-wrap.MuiTypography-body1')
        tmp = []
        tmp_review = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        for i in range(len(beer_review)):
            tmp_review.append(beer_review[i].text)
            # print(tmp_review.append(beer_review[i].text))
            
        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['검색이름'] = beer
        tmp['beer_review'] = tmp_review

        data = pd.concat([data, tmp])
        # data = pd.concat([data, tmp_review], axis = 1).reset_index(inplace=True, drop=True)

        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(2)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result

In [12]:
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

url_open... Berliner Kindl Pilsener 맥주 데이터를 수집합니다..
Berliner Kindl Jubiläums Pilsener Premium
Reviews (307)
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
리뷰수 :  307 수집된 리뷰수 :  307


# 여기부터 합니다 11/10

In [171]:
beer_list = ['kloud', 'fitz super clear', 'Asahi super dry', 'Tsingtao', 'Heineken',
 'Kirin ichiban', 'Sapporo Premium Beer / Draft Beer', 'stella artois', 'guinness braught',
 '1664 Blanc', 'pilsner urquell', 'San Miguel','OB premier pilsner', 'cass fresh',
 'Hite Black Beer Stout', 'dry finish', 'max hite', 'hite extra cold', 'victoria bitter', 'BINTANG pilsner',
 'krombacher weizen', 'Miller Genuine Draft', 'Hoegaarden Cherry', 'TIGER REDLER',
 "Suntory The Premium Malt's", 'REEPER B Weissbier', 'PEEPER B IPA', 'TIGER BEER',
 'TSINGTAO WHEAT BEER', 'Erdinger Weissbier', 'Carlsberg', 'Budweiser ', 'Hoegaarden',
 'YEBISU', 'Paulaner Hefe-weissbier', 'Desperados', 'Peroni Nastro Azzurro',
 'Edelweiss Snowfresh', 'Heineken Dark Lager', 'Kozel Dark 10', 'Guinness original',
 'Filite', 'SEOULITE ALE', 'JEJU WIT ALE', 'Stephans Brau Philsner', 'Stephans Brau Larger',
 'Stephans Bräu Hefe-Weizen Naturtrüb', 'Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type", 'Terra', 
 'Hite Zero', 'HiteJinro Filite Fresh Cool & Clear', 'Cass Light', 'Happoshu Filgood', 
 'Kloud Draft Original Gravity', 'Hite Beer', '7Bräu Gompyo Wheatbeer', 
 'Moonbear Geumgangsan Golden Ale','Blue Moon Belgian White', 'Heineken Silver', 'Cafri']

In [125]:
beer_list[76]

'Hite Zero'

In [1]:
#######맥주 국가, 종류 추가 크롤링######

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name', 'beer_country', 'beer_type', 'beer_alcohol'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)

    # 2번 사진에 해당 : 상품 선택
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # 4번 사진에 해당 : 전체 리뷰 개수 수집
            time.sleep(3)
            string = driver.find_element_by_class_name('MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    time.sleep(3)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    driver.execute_script("arguments[0].click();", element)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        beer_info = driver.find_elements_by_css_selector('.px-4.fj-s.f-wrap')

        tmp = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])

        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element_by_xpath('//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result

SyntaxError: invalid syntax (1254915663.py, line 3)

In [137]:
beer_list = ['hite zero']

In [173]:
# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

# data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name', 'beer_country', 'beer_type', 'beer_alcohol'])
data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name'])

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    # data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name', 'beer_country', 'beer_type', 'beer_alcohol'])
    data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name'])

    # url open
    print(k, 'url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #     options = webdriver.ChromeOptions()
    #     options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    # 명시적 대기 - XPATH를 이용한 경우
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')))
    #time.sleep(4)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input').send_keys(beer)
    #time.sleep(4)

    # 2번 사진에 해당 : 상품 선택
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]')))
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4')))
    beer_name = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    error_cnt = 0
    
    beer_country = driver.find_element(By.CSS_SELECTOR,'.hwjOn.MuiTypography-body2>span').text
    beer_type = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.kbrPIo.colorized__WrappedComponent-hrwcZr.liJcHu.Anchor___StyledText-uWnSM.eseQug.MuiTypography-caption').text
    
    try:
        beer_alcohol = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.kbrPIo.colorized__WrappedComponent-hrwcZr.bRPQdN.MuiTypography-caption').text
    except:
        print('none')
    
    tmp = []
    tmp_beer_country = []
    tmp_beer_type = []
    tmp_beer_alcohol = []

    tmp_beer_country.append(beer_country)
    tmp_beer_type.append(beer_type)

    try:
        tmp_beer_alcohol.append(beer_alcohol)
    except:
        tmp_beer_alcohol.append('none')


    tmp = pd.DataFrame(data=tmp_beer_country, columns=['beer_country'])

    tmp['beer_name'] = beer_name
    tmp['검색이름'] = beer
    tmp['beer_country'] = tmp_beer_country
    tmp['beer_type'] = tmp_beer_type
    tmp['beer_alcohol'] = tmp_beer_alcohol

    data = pd.concat([data, tmp])

    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("./beer_info_data/beer_n_"+str(k)+".csv", encoding='utf-8')

    driver.quit()
    return result

In [174]:
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

0 url_open... kloud 맥주 데이터를 수집합니다..
1 url_open... fitz super clear 맥주 데이터를 수집합니다..
2 url_open... Asahi super dry 맥주 데이터를 수집합니다..
3 url_open... Tsingtao 맥주 데이터를 수집합니다..
4 url_open... Heineken 맥주 데이터를 수집합니다..
5 url_open... Kirin ichiban 맥주 데이터를 수집합니다..
6 url_open... Sapporo Premium Beer / Draft Beer 맥주 데이터를 수집합니다..
7 url_open... stella artois 맥주 데이터를 수집합니다..
8 url_open... guinness braught 맥주 데이터를 수집합니다..
9 url_open... 1664 Blanc 맥주 데이터를 수집합니다..
10 url_open... pilsner urquell 맥주 데이터를 수집합니다..
11 url_open... San Miguel 맥주 데이터를 수집합니다..
12 url_open... OB premier pilsner 맥주 데이터를 수집합니다..
13 url_open... cass fresh 맥주 데이터를 수집합니다..
14 url_open... Hite Black Beer Stout 맥주 데이터를 수집합니다..
15 url_open... dry finish 맥주 데이터를 수집합니다..
16 url_open... max hite 맥주 데이터를 수집합니다..
17 url_open... hite extra cold 맥주 데이터를 수집합니다..
18 url_open... victoria bitter 맥주 데이터를 수집합니다..
19 url_open... BINTANG pilsner 맥주 데이터를 수집합니다..
20 url_open... krombacher weizen 맥주 데이터를 수집합니다..
21 url_open... Miller Genuine Draft 맥주 데이터를 수집합

In [175]:
result

,검색이름,beer_name,beer_country,beer_type,beer_alcohol
0,Cafri,Cafri,🇰🇷,Pale Lager - International / Premium,4.2%


In [169]:
result.loc[result['beer_country'] == '🇰🇷']

,검색이름,beer_name,beer_country,beer_type,beer_alcohol
0,hite zero,Hite Zero,🇰🇷,Low / No Alcohol Beer - Pale,none


In [ ]:
#######국가, 맥주종류, 도수 ######



beer_list = ['Berliner Kindl Pilsener']

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

# data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name', 'beer_country', 'beer_type', 'beer_alcohol'])
data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name'])

def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    # data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name', 'beer_country', 'beer_type', 'beer_alcohol'])
    data = pd.DataFrame(data=[], columns=['검색이름', 'beer_name'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    #     options = webdriver.ChromeOptions()
    #     options.add_argument('headless')
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.set_window_size(900, 900)

    # 1번 사진에 해당 : 맥주 검색
    # 명시적 대기 - XPATH를 이용한 경우
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')))
    #time.sleep(4)
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input').send_keys(beer)
    #time.sleep(4)

    # 2번 사진에 해당 : 상품 선택
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]')))
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4')))
    beer_name = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    error_cnt = 0
    beer_country = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    beer_type = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    beer_alcohol = driver.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    print(beer_name, beer_country, beer_type, beer_alcohol)
    while 1:
        try :
            # 알콜도수 수집
            time.sleep(4)
            # elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6')))
            string = driver.find_element(By.CLASS_NAME,'MuiTypography-root Text___StyledTypographyTypeless-bukSfn kbrPIo colorized__WrappedComponent-hrwcZr liJcHu Anchor___StyledText-uWnSM eseQug MuiTypography-caption').text
            print(string)

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    #time.sleep(4)
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')))
    # element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    #time.sleep(4)

    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    driver.execute_script("arguments[0].click();", elem)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')
        # Show more
        #driver.execute_script("arguments[0].click();", element)
        show_more = driver.find_elements(By.CSS_SELECTOR,'section.BeerReviewListItem___StyledSection2-gzaqoT.JrTsK > div.fj-s.f-wrap > div > button > span.MuiButton-label')
        for i in range(len(show_more)):
            driver.execute_script("arguments[0].click();", show_more[i])

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        # 맥주 정보
        beer_info = driver.find_elements(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap')
        beer_review = driver.find_elements(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.colorized__WrappedComponent-hrwcZr.hwjOn.BeerReviewListItem___StyledText-kMbsdb.gCtEHi.pre-wrap.MuiTypography-body1')
        tmp = []
        tmp_review = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        for i in range(len(beer_review)):
            tmp_review.append(beer_review[i].text)
            # print(tmp_review.append(beer_review[i].text))
            
        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['검색이름'] = beer
        tmp['beer_review'] = tmp_review

        data = pd.concat([data, tmp])
        # data = pd.concat([data, tmp_review], axis = 1).reset_index(inplace=True, drop=True)

        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(2)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result

In [11]:
driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[1]/div[1]/div/div[2]/div[1]/div/div[2]/div[3]/div[1]')

AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'